In [ ]:
import pandas as pd
import pymysql
import getpass

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [ ]:
conn = pymysql.connect(host="35.233.174.193",port=3306,
                       user="jovyan",passwd=getpass.getpass("Enter password for MIMIC2 database"),
                       db='mimic2')

# Administrative and demographic data
Administrative data defines general information about the patient, such as:
- Name
- Sex
- Date of birth
- Insurance information

In MIMIC, patient data is stored in a table called `d_patients`. Additional demograpic data is stored in a table called `demographic_detail`.

Let's first select all (`"select *"`) for the first 5 patients in `d_patients`:

In [ ]:
query = """
select * from d_patients limit 5;
"""
df = pd.read_sql(query, conn)
df

We can look at data for a specific patient by using a `where` statement to filter to a specific subject id:

In [ ]:
query = """
select * from d_patients 
where subject_id = 31;
"""
df = pd.read_sql(query, conn)
df

Lets look at what's in `demographic_detail`:

In [ ]:
query = """
select * from demographic_detail limit 5;
"""
df = pd.read_sql(query, conn)
df

We can join these two tables using a `join` statement. The column `subject_id`, which is the identifier for a patient, is consistent between these two columns and can be used to join them together:

In [ ]:
query = """
select * 
from d_patients
    join demographic_detail on d_patients.subject_id = demographic_detail.subject_id
limit 5;
"""
df = pd.read_sql(query, conn)
df

Now, let's see what we can do with this data. Let's say that we want to know what age patients were when they died. To do this, we can use a function to subtract `dob` ("date of birth") from `dod` ("date of death"):

In [ ]:
query = """
select subject_id, sex, 
    dob, dod, 
    floor(datediff(dod, dob) / 365)  as 'age_at_death'
from mimic2.d_patients
limit 100;
"""
df = pd.read_sql(query, conn)
df

Maybe we're interested in the max, min, and average ages. We can use agreggate functions to do this:

In [ ]:
query = """
select
    count(1) as 'number_of_patients',
    max(floor(datediff(dod, dob) / 365))  as 'max_age_at_death',
    min(floor(datediff(dod, dob) / 365))  as 'min_age_at_death',
    avg(floor(datediff(dod, dob) / 365))  as 'avg_age_at_death'
    
from (select dod, dob from mimic2.d_patients limit 1000) sub;
"""
df = pd.read_sql(query, conn)
df

Now, let's use Python to do something similar. Let's query data and use Python to calculate these statistics and then plot the distribution:

In [ ]:
# First, query the raw data
query = """
select subject_id, sex, 
    dob, dod, 
    floor(datediff(dod, dob) / 365)  as 'age_at_death'
from mimic2.d_patients
limit 1000;
"""
df = pd.read_sql(query, conn)
df

In [ ]:
# Summary statistics
df['age_at_death'].describe()

Now, let's use some additional Python libraries to plot this data in a histogram:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ax = sns.distplot(df['age_at_death'])

In [ ]:
# We can also use a boxplot:
ax = sns.boxplot( y='age_at_death', data=df)

Let's say we also want to compare the distribution of age of death between men and women. We can do this in a boxplot by adding an 'x' variable:

### Discussion
Looking at these two plots, what can you say about the difference between the age of death for men and women?

In [ ]:
ax = sns.boxplot(x='sex', y='age_at_death', data=df, order=['F', 'M'])

In [ ]:
_ = df.hist('age_at_death', by='sex', sharey=True, sharex=True)

# Diagnoses
The table `icd9` contains the conditions which patients are assigned diagnoses with. ICD-9 codes are a standardized terminology where each code represents a very specific disease. ICD-9 has been replaced with ICD-10 coding, but historical data like MIMIC still contains ICD-9. Next week we'll discuss in more detail what these codes signify. For now, let's just look at a few examples to see what diagnoses are in the database:

In [ ]:
query = """
select * from icd9
limit 10;
"""
df = pd.read_sql(query, conn)
df.head(10)

### TODO
Let's look at the diagnoses for a specific patient. Write a query below to return the ICD-9 code and description for patient **286**.

In [ ]:
query = """
select * from icd9
where ___ = ___;
"""
df = pd.read_sql(query, conn)
df

Next, let's look not just at a single patient but the entire database. Let's write a query which will count how many times an ICD-9 code occurs for any patient:

### TODO
Change the query below so that we group by code and description and limit to the 10 most frequent diagnoses.

In [ ]:
# What are the 10 most frequent diagnoses codes?
query = """
select code, description, count(*)
from icd9
group by ___, ___
order by count(*) desc
limit ___;
"""
df = pd.read_sql(query, conn)
df

Now let's find the patients who have these 10 codes. Let's save this list of 10 codes as a Python variable and then we will add this into our query using the `.format()` method in Python:

In [ ]:
top_10_codes = tuple(df['code'])
top_10_codes

In [ ]:
query = """
select distinct subject_id, code
from icd9
where code in {0}
""".format(top_10_codes)
print(query)

In [ ]:
# Now let's run the query
df = pd.read_sql(query, conn)
df.head()

In [ ]:
len(df)

### TODO
Let's plot a barplot of the 10 most common ICD-9 codes in the database.

- Write a query to return the code, description, and count of each ICD-9 code (hint: you'll want to use a `group by` statement
- Limit this query to the 10 most common codes
- Store the results of the query in a variable called `top_10_df`
- Call the `sns.barplot` method to plot a barplot. The x axis should be the ICD-9 codes and the y axis should be the counts

In [ ]:
query = """
select ___, ___, ___(*) as 'count'
from icd9
___ ___ code, description
order by count(*) desc
___ ___;
""".format(top_10_codes)
top_10_df = pd.read_sql(query, conn)
top_10_df

In [ ]:
sns.___(x=___, y=___, data=top_10_df, ci=None)

## Comorbidities
We're often interested in knowing about the "comorbidity" of a disease. A comorbidity is a condition which a patient has in addition to another condition. For example, if a patient has diabetes and they are also diagnoses with hypertension, then these two conditions would be comorbid. 

This is useful if we want to understand what conditions a population of patients might be at risk for based on the conditions they already have, or for measuring how certain diseases interact.

In this exercise we will calculate how frequently the 10 most common ICD-9 codes co-occur together. Because this is somewhat complicated, I've already generated the comorbidity statistics. We'll look at how to interpret this information, and for homework you'll answer some specific questions about this data.

In [ ]:
import helpers

In [ ]:
from importlib import reload
reload(helpers)

In [ ]:
co_mtrx = helpers.create_co_mtrx(conn)
co_mtrx

In [ ]:
fig, ax = helpers.plot_co_mtrx(co_mtrx)

### TODO
For the first 3 diagnoses (250.00 - 428.0), identify what condition most frequently co-occurs with them. Write them in this markdown cell.

- 250.00, "DIABETES MELLITUS WITHOUT COMPLICATION TYPE II." -> ... 
- 401.9, "UNSPECIFIED ESSENTIAL HYPERTENSION" -> ...
- 414.01, "CORONARY ATHEROSCLEROSIS OF NATIVE CORONARY AR" -> ...